In [1]:
import pandas as pd
import numpy as np
import sklearn

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv('featuresTrain2.csv')
test = pd.read_csv('featuresTest2.csv')

In [3]:
train.head()

,id,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,...,centroscomercialescercanos,precio,anio,mes,dia,datoBienCargado,peso_IDF_descripcion,peso_IDF_titulo,metrosdescubiertos,tipodepropiedad_cb
0,254099,Benito Juárez,Distrito Federal,2.210044,2.0,1.0,2.0,4.394449,4.394449,23533.0,...,0.0,2273000.0,2015.0,8.0,23.0,1,0.086408,0.000000,0.000000,0.478679
1,53461,La Magdalena Contreras,Distrito Federal,2.397895,3.0,2.0,2.0,5.594711,5.198497,24514.0,...,1.0,3600000.0,2013.0,6.0,28.0,0,0.344349,0.165371,0.000000,0.386072
2,247984,Tonalá,Jalisco,1.791759,3.0,2.0,2.0,4.976734,5.117994,48551.0,...,0.0,1200000.0,2015.0,10.0,17.0,1,0.992218,0.033601,3.135494,0.426561
3,209067,Zinacantepec,Edo. de México,0.693147,2.0,1.0,1.0,4.158883,4.219508,53666.0,...,1.0,650000.0,2012.0,3.0,9.0,1,0.107927,0.033601,1.609438,0.426561
4,185997,Zapopan,Jalisco,2.397895,2.0,1.0,1.0,4.564348,4.564348,47835.0,...,0.0,1150000.0,2016.0,6.0,7.0,1,0.078086,0.000000,0.000000,0.478679


In [4]:
y = train['precio']
train = train.drop(['id','precio','ciudad','provincia'],1)
train.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,gimnasio,...,escuelascercanas,centroscomercialescercanos,anio,mes,dia,datoBienCargado,peso_IDF_descripcion,peso_IDF_titulo,metrosdescubiertos,tipodepropiedad_cb
0,2.210044,2.0,1.0,2.0,4.394449,4.394449,23533.0,20.696818,-99.509221,0.0,...,0.0,0.0,2015.0,8.0,23.0,1,0.086408,0.000000,0.000000,0.478679
1,2.397895,3.0,2.0,2.0,5.594711,5.198497,24514.0,19.310205,-99.227655,0.0,...,1.0,1.0,2013.0,6.0,28.0,0,0.344349,0.165371,0.000000,0.386072
2,1.791759,3.0,2.0,2.0,4.976734,5.117994,48551.0,20.696818,-99.509221,0.0,...,0.0,0.0,2015.0,10.0,17.0,1,0.992218,0.033601,3.135494,0.426561
3,0.693147,2.0,1.0,1.0,4.158883,4.219508,53666.0,19.301890,-99.688015,0.0,...,1.0,1.0,2012.0,3.0,9.0,1,0.107927,0.033601,1.609438,0.426561
4,2.397895,2.0,1.0,1.0,4.564348,4.564348,47835.0,20.696818,-99.509221,0.0,...,0.0,0.0,2016.0,6.0,7.0,1,0.078086,0.000000,0.000000,0.478679


In [5]:
y.head()

0    2273000.0
1    3600000.0
2    1200000.0
3     650000.0
4    1150000.0
Name: precio, dtype: float64

In [6]:
# primero voy a definir un xgboost con valores promerios para poder tunearlo:
modelo_rf = RandomForestRegressor(
    n_estimators=1000,
    max_depth=3,
    min_samples_leaf=5,
    n_jobs = -1,
    random_state =5 ,
    max_features = "auto"
)

In [7]:
#Entreno dicho xgboost:
modelo_rf.fit(train,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=5, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=-1, oob_score=False, random_state=5, verbose=0,
                      warm_start=False)

In [20]:
# esto nos ayudara a tunear el algoritmo, la idea es que tunearlo con los valores de la lista:
parameters_for_testing = {
    'max_depth':[1000],
    'min_samples_leaf':[15],
    'n_estimators':[1000]
}

In [21]:
# defino el gridsearch y lo aplico en el xgboost que defini para tunearlo:

gsearch1 = GridSearchCV(estimator = modelo_rf, param_grid = parameters_for_testing,\
                        n_jobs=4,cv=3,iid=False, verbose=10,scoring='neg_mean_absolute_error',refit=True)
gsearch1.fit(train,y)
#print (gsearch1.grid_scores_)
print('best params')
print (gsearch1.best_params_)
print('best score')
print (gsearch1.best_score_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed: 22.4min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed: 22.4min finished


best params
{'max_depth': 100, 'min_samples_leaf': 15, 'n_estimators': 1000}
best score
-655897.371704433


In [17]:
# con los datos del gridsearch defino el mejor modelo de xgboost que encontre:
mejor_xgb =  xgboost.XGBRegressor(
     learning_rate =0.1,
     n_estimators=1000,
     max_depth=7,
     min_child_weight=3,
     gamma=0.5,
     subsample=0.95,
     colsample_bytree=0.8,
     objective= 'reg:squarederror',
     nthread=4,
     scale_pos_weight=1)

NameError: name 'xgboost' is not defined

In [10]:
#Entreno el modelo:
mejor_xgb.fit(train,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0.5,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=7, min_child_weight=3, missing=None, n_estimators=1000,
             n_jobs=1, nthread=4, objective='reg:squarederror', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.95, verbosity=1)

In [11]:
ids = test['id']
test = test.drop('id',axis = 1)

In [12]:
# realizo la prediccion:
prediction = mejor_xgb.predict(test)

In [13]:
# creo el dataframe para la sumision:
dataKaggle = pd.DataFrame(ids ,columns={'id'})

In [14]:
dataKaggle['target'] = prediction

In [15]:
dataKaggle.head()

,id,target
0,4941,5.082817e+06
1,51775,1.202061e+06
2,115253,1.989034e+06
3,299321,1.396003e+06
4,173570,5.981472e+05


In [16]:
dataKaggle.to_csv("predictionXGBoost.csv", index = False)

In [ ]:
# 566214.13293

In [ ]:
#mejor modelo xgboost:
# con los datos del gridsearch defino el mejor modelo de xgboost que encontre:
#mejor_xgb =  xgboost.XGBRegressor(
#     learning_rate =0.1,
#     n_estimators=1000,
#     max_depth=7,
#     min_child_weight=3,
#     gamma=0.5,
#     subsample=0.95,
#     colsample_bytree=0.8,
#     objective= 'reg:squarederror',
#     nthread=4,
#     scale_pos_weight=1)[mejor1]

In [55]:
test.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,anio,mes,dia,datoBienCargado,peso_IDF_descripcion,peso_IDF_titulo,provincia_cb,tipodepropiedad_cb
0,29.0,3.0,NaN,4.0,300.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2013,7,20,0,0.018539,0.033601,0.404545,0.426768
1,NaN,1.0,1.0,1.0,67.0,67.0,113851.0,0.0,0.0,0.0,0.0,0.0,2015,10,24,1,0.714877,0.000000,0.567188,0.480004
2,0.0,2.0,1.0,2.0,87.0,100.0,23620.0,0.0,0.0,0.0,0.0,1.0,2015,5,30,1,0.530057,0.638628,0.404545,0.480004
3,2.0,2.0,2.0,2.0,86.0,86.0,129347.0,0.0,0.0,0.0,0.0,0.0,2015,4,2,1,0.296724,0.091081,0.364473,0.480004
4,10.0,2.0,1.0,1.0,80.0,76.0,57125.0,0.0,0.0,0.0,1.0,1.0,2013,8,15,0,0.619535,0.033601,0.422059,0.426768
